# Лабораторная №1

## Вариант 1

In [15]:
import numpy as np
from scipy.optimize import fsolve

Мы предполагаем, что имеется 3 состояния в момент 1. В нашей модели имеются три агента, и каждый из них имеет начальный капитал в 16 единиц в момент 0 и 50 единиц в момент 1 в каждом состоянии.

In [14]:
e0 = 16  # Начальный капитал в t=0
e1 = 50  # Капитал в t=1

Все агенты имеют одинаковые функции полезности вида $$U(c,C)=\sqrt c +\sqrt C.$$

In [17]:
def u(c):
    return np.sqrt(c)

Агенты имеютразличные субъективные вероятности состояний в момент 1

In [16]:
P = np.array([
    [0.5, 0.25, 0.25],
    [0.25, 0.5, 0.25],
    [0.25, 0.25, 0.5]
])

Требуется

1. Найти оптимальное распределение потребления агентов в условиях равновесия.

2. Найти безрисковую процентную ставку.

Итак, у нас имеется 15 неизвестных:

- $c_1, c_2, c_3$ -- текущее потребление;

- $C_{11}, C_{12}, C_{13}, C_{21},\ldots, C_{33}$ -- потребления в состояниях;

- $\psi_1, \psi_2, \psi_3$ -- цены состояний.

Для их вычисления мы составим систему из 15 уравнений. Уравнения будут следующих типов:

1. **Условия первого порядка.** Для каждого агента $i$ и состояния $j$ выполняется $$/dfrac{P_{ij} \cdot U(c_i)}{U(C_{ij}} = \psi_j,\ i = \overline{1,3},\ j = \overline{1,3}.$$ Эти 9 уравнений выводятся из условия максимизации полезности с учётом вероятностей состояний и соответствующих цен состояний.

2. **Бюджетные ограничения.** Для каждого агента $i$ выполняется $$c_i + \sum_{j=1}^3\psi _j C_{ij} = e_0 + e_1\cdot \sum _{j=1}^3 \psi_j.$$ Эти 3 уравнения отражают баланс между текущим потреблением и будущими затратами, скорректированными на цены состояний.

3. **Рыночное равновесие.** Для каждого состояния $j$ $$\sum_{j=1}^3 C_{ij} = 3e_1.$$ Это уравнение гарантирует, что общее потребление в каждом состоянии равно общему доступному капиталу. 

In [18]:
def equations(vars):
    c = vars[:3]               # c1, c2, c3 - потребление агентов
    C = vars[3:12].reshape(3, 3)  # C11, C12, ..., C33 - потребление в состояниях для агентов
    psi = vars[12:]              # psi1, psi2, psi3 - цены состояний

    eqs = []

    # Условия первого порядка
    for i in range(3): # по агентам
        for s in range(3): # по состояниям
            eqs.append(P[i, s] * u(c[i]) / u(C[i, s]) - psi[s])

    # Бюджетные ограничения для каждого агента
    for i in range(3):
        eqs.append(c[i] + np.dot(psi, C[i, :]) - (e0 + e1 * np.sum(psi)))

    # Рыночное равновесие
    for s in range(3):
        eqs.append(np.sum(C[:, s]) - 3 * e1)

    return eqs

initial_guess = np.ones(15) # Начальное приближение
solution = fsolve(equations, initial_guess) # Решение системы

In [19]:
# Разбиваем результаты
c_sol = solution[:3]            # Равновесное текущее потребление
C_sol = solution[3:12].reshape(3, 3)  # Равновесное потребление в состояниях
psi_sol = solution[12:]           # Цены состояний

print("Текущее потребление (с):", c_sol, end="\n\n")
print("Потребления в состояниях (С):")
print(C_sol, end="\n\n")
print("Цены  для состояний (psi):", psi_sol, end="\n\n")

Текущее потребление (с): [16. 16. 16.]

Потребления в состояниях (С):
[[100.  25.  25.]
 [ 25. 100.  25.]
 [ 25.  25. 100.]]

Цены  для состояний (psi): [0.2 0.2 0.2]



Безрисковая процентная ставка $r_f$ рассчитывается как:
$$r_f = \dfrac{1}{\sum_{j=1}^3 \psi_j} -1.$$
Эта ставка показывает доходность безрискового актива в условиях равновесия.

In [20]:
r_f = 1 / np.sum(psi_sol) - 1  # Безрисковая процентная ставка
print("Безрисковая процентная ставка (r_f):", r_f)

Безрисковая процентная ставка (r_f): 0.6666666666666397


## Вариант 2

Все агенты имеют одинаковые функции полезности вида $$U(c,C)= c-\dfrac{c^2}{340} + C-\dfrac{C^2}{340}.$$

In [21]:
def u(c):
    return c - c**2 / 340